# Mapy animowane

***Autor: Adam Dąbkowski***

## 0. Importowanie niezbędnych bibliotek

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from matplotlib.animation import FuncAnimation
from shapely.geometry import Point, LineString
from typing import List

## 1. Wczytywanie zbioru danych

Źródło danych: ***https://globalfishingwatch.org/data-download/datasets/public-training-data-v1/***

In [2]:
DATASET_PATH: str = "ships_trajectories.csv"

In [3]:
df = pd.read_csv(DATASET_PATH, low_memory=False)
df.head(n=100)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
0,1.848346e+13,1.340882e+09,0.000000,2236.013184,0.0,0.000000,28.967354,-13.537797,-1.0,gfw
1,1.848346e+13,1.340884e+09,0.000000,2236.013184,0.0,125.199997,28.967373,-13.537838,-1.0,gfw
2,1.848346e+13,1.340885e+09,0.000000,2236.013184,0.0,0.000000,28.967354,-13.537838,-1.0,gfw
3,1.848346e+13,1.340888e+09,0.000000,2236.013184,0.0,0.000000,28.967354,-13.537838,-1.0,gfw
4,1.848346e+13,1.340925e+09,1999.950928,2828.357666,8.7,203.100006,28.929653,-13.543955,-1.0,gfw
...,...,...,...,...,...,...,...,...,...,...
95,1.848346e+13,1.342261e+09,26400.109375,57077.492188,8.4,220.600006,28.525244,-14.365098,-1.0,gfw
96,1.848346e+13,1.342265e+09,38274.378906,75536.218750,9.6,228.100006,28.431360,-14.516421,-1.0,gfw
97,1.848346e+13,1.342271e+09,35354.472656,80278.781250,10.3,222.699997,28.302937,-14.719269,-1.0,gfw
98,1.848346e+13,1.342276e+09,48506.542969,55035.000000,8.5,244.699997,28.167040,-14.928179,-1.0,gfw


## 2. Wybór statku

#### 2.1 Region

##### 2.1.1 USA

In [4]:
# SHIP_ID: float = 47334168288729.0
# MAP_LONGITUDE_RANGE: List[int] = [-80, -60]
# MAP_LATITUDE_RANGE: List[int] = [30, 50]

##### 2.1.2 Japonia I

In [5]:
# SHIP_ID: float = 196184437994412.0
# MAP_LONGITUDE_RANGE: List[int] = [120, 155]
# MAP_LATITUDE_RANGE: List[int] = [15, 50]

##### 2.1.3 Japonia II

In [6]:
SHIP_ID: float = 102029098096261.0
MAP_LONGITUDE_RANGE: List[int] = [120, 175]
MAP_LATITUDE_RANGE: List[int] = [10, 55]

##### 2.1.4 Afryka Zachodnia I

In [7]:
# SHIP_ID: float = 18483455048454.0
# MAP_LONGITUDE_RANGE: List[int] = [-25, -5]
# MAP_LATITUDE_RANGE: List[int] = [20, 35]

##### 2.1.5 Afryka Zachodnia II

In [8]:
# SHIP_ID: float = 214572731257568.0
# MAP_LONGITUDE_RANGE: List[int] = [-45, 20]
# MAP_LATITUDE_RANGE: List[int] = [20, 55]

#### 2.2 Selekcja danych

In [9]:
df_ship = df[df["mmsi"] == SHIP_ID]
df_ship.head(n=100)

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,is_fishing,source
130059,1.020291e+14,1.429227e+09,999.975464,28318.910156,0.0,0.0,34.867195,138.328293,-1.0,gfw
130060,1.020291e+14,1.429228e+09,999.975464,28318.910156,0.0,0.0,34.867188,138.328308,-1.0,gfw
130061,1.020291e+14,1.429231e+09,999.975464,28318.910156,0.0,0.0,34.867199,138.328262,-1.0,gfw
130062,1.020291e+14,1.429232e+09,999.975464,28318.910156,0.0,0.0,34.867195,138.328262,-1.0,gfw
130063,1.020291e+14,1.429235e+09,999.975464,28318.910156,0.0,0.0,34.867195,138.328247,-1.0,gfw
...,...,...,...,...,...,...,...,...,...,...
130154,1.020291e+14,1.429493e+09,999.975464,28318.910156,0.0,0.0,34.867195,138.328262,-1.0,gfw
130155,1.020291e+14,1.429494e+09,999.975464,28318.910156,0.0,0.0,34.867172,138.328247,-1.0,gfw
130156,1.020291e+14,1.429496e+09,999.975464,28318.910156,0.0,0.0,34.867180,138.328247,0.0,gfw
130157,1.020291e+14,1.429497e+09,999.975464,28318.910156,0.0,0.0,34.867180,138.328247,0.0,gfw


## 3. Wizualizacja danych

In [10]:
WORD_MAP_PATH: str = "ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp"
REFRESH_FREQUENCY: int = 100

#### 3.1 Wizualizacja trasy statku na mapie

In [ ]:
matplotlib.use('Qt5Agg')
plt.style.use('seaborn-v0_8-whitegrid')

points = [Point(row["lon"], row["lat"]) for _, row in df_ship.iterrows()]
timestamps = pd.to_datetime(df_ship["timestamp"], unit='s')
gdf_points = gpd.GeoDataFrame(geometry=points)

line = LineString(points)
gdf_line = gpd.GeoDataFrame(geometry=[line])

world_map = gpd.read_file(WORD_MAP_PATH)

fig, ax = plt.subplots(figsize=(10, 6))
world_map.plot(ax=ax, color='lightgray')

ax.set_xlim(MAP_LONGITUDE_RANGE)
ax.set_ylim(MAP_LATITUDE_RANGE)

trail, = ax.plot([], [], color='#ff6361', linewidth=2, label="Trasa statku")
current_pos, = ax.plot([], [], 'o', color='#58508d', label="Pozycja statku")

date_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

def init():
    trail.set_data([], [])
    current_pos.set_data([], [])
    date_text.set_text('')
    return trail, current_pos, date_text

def update(frame):
    x_coords = [p.x for p in points[:frame + 1]]
    y_coords = [p.y for p in points[:frame + 1]]
    trail.set_data(x_coords, y_coords)
    current_pos.set_data([x_coords[-1]], [y_coords[-1]])
    current_time = timestamps.iloc[frame].strftime('%Y-%m-%d %H:%M:%S')
    date_text.set_text(f'Data: {current_time}')
    return trail, current_pos, date_text

ani = FuncAnimation(fig, update, frames=range(len(points)), init_func=init, interval=(1000/REFRESH_FREQUENCY), blit=True)

ax.legend()
ax.set_title("Animowana trasa statku")

plt.show()